# 第5章: Claudeの出力フォーマットと発話

- [レッスン](#lesson)
- [演習](#exercises)
- [例のプレイグラウンド](#example-playground)

## セットアップ

次のセットアップセルを実行して、APIキーをロードし、`get_completion`ヘルパー関数を確立します。

In [ ]:
# Pythonの組み込み正規表現ライブラリをインポート
import re
import boto3
from botocore.exceptions import ClientError
import json

# utilsパッケージからhintsモジュールをインポート
from utils import hints

# IPythonストアからMODEL_NAME変数を取得
%store -r modelId
%store -r region

bedrock_client = boto3.client(service_name='bedrock-runtime', region_name=region)

In [ ]:
def get_completion(prompt, system_prompt=None, prefill=None):
    inference_config = {
        "temperature": 0.5,
        "maxTokens": 200
    }
    converse_api_params = {
        "modelId": modelId,
        "messages": [{"role": "user", "content": [{"text": prompt}]}],
        "inferenceConfig": inference_config
    }
    if system_prompt:
        converse_api_params["system"] = [{"text": system_prompt}]
    if prefill:
        converse_api_params["messages"].append({"role": "assistant", "content": [{"text": prefill}]})
    try:
        response = bedrock_client.converse(**converse_api_params)
        text_content = response['output']['message']['content'][0]['text']
        return text_content

    except ClientError as err:
        message = err.response['Error']['Message']
        print(f"クライアントエラーが発生しました: {message}")

---

## レッスン

**Claudeはその`output`をさまざまな方法でフォーマットできます**。そうするように頼むだけです！

その方法の一つは、XMLタグを使用して、応答を他の余分なテキストから分離することです。すでに、XMLタグを使用してプロンプトをより明確で解析しやすくする方法を学びました。実際、Claudeに**XMLタグを使用してその`output`をより明確で人間にとって理解しやすくするように頼むこともできます**。

### 例

第2章で解決した「詩の前置き問題」を思い出してください。Claudeに前置きを完全にスキップするように頼んだことがありましたが、**Claudeに詩をXMLタグで囲むように指示することでも、同様の結果を得ることができる**ことがわかりました。

In [ ]:
# 変数の内容
ANIMAL = "Rabbit"

# 変数の内容のためのプレースホルダーを持つプロンプトテンプレート
PROMPT = f"Please write a haiku about {ANIMAL}. Put it in <haiku> tags."

# Claudeの応答を出力
print("--------------------------- 変数の置換を含む完全なプロンプト ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claudeの応答 -------------------------------------")
print(get_completion(PROMPT))

なぜこれを行いたいのでしょうか？それは、**出力がXMLタグ内にあることで、エンドユーザーが短いプログラムを書いてXMLタグの間のコンテンツを抽出することで、詩を確実に取得できるからです**。

この技術の拡張として、**最初のXMLタグを`assistant`ターンに置くことがあります。`assistant`ターンにテキストを置くと、基本的にClaudeに何かを言ったことを伝え、そのポイントから続けるべきだと指示していることになります。この技術は「Claudeのために話す」または「Claudeの応答を事前に埋める」と呼ばれています**。

以下では、最初の`<haiku>` XMLタグを使ってこれを行いました。Claudeが私たちがやめたところから直接続ける様子に注目してください。

In [ ]:
# 変数の内容
ANIMAL = "Cat"

# 変数の内容のためのプレースホルダーを持つプロンプトテンプレート
PROMPT = f"Please write a haiku about {ANIMAL}. Put it in <haiku> tags."

# Claudeの応答のためのプレフィル
PREFILL = "<haiku>"

# Claudeの応答を出力
print("--------------------------- 変数置換を含む完全なプロンプト ---------------------------")
print("ユーザーのターン:")
print(PROMPT)
print("\nアシスタントのターン:")
print(PREFILL)
print("\n------------------------------------- Claudeの応答 -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

Claudeは他の出力フォーマットスタイル、特に`JSON`を使用するのも得意です。`JSON`出力を強制したい場合（決定的ではありませんが、近い形で）、Claudeの応答を開きかっこ`{`で事前に埋めることもできます。

In [ ]:
# Variable content
ANIMAL = "Cat"

# プロンプトテンプレート、変数の内容のためのプレースホルダー付き
PROMPT = f"Please write a haiku about {ANIMAL}. Use JSON format with the keys as \"first_line\", \"second_line\", and \"third_line\"."

# Claudeの応答のためのプレフィル
PREFILL = "{"

# Claudeの応答を出力します
print("--------------------------- 変数置換を含む完全なプロンプト ---------------------------")
print("ユーザーのターン")
print(PROMPT)
print("\nアシスタントのターン")
print(PREFILL)
print("\n------------------------------------- Claudeの応答 -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

以下は、**同じプロンプト内での複数の入力変数と出力フォーマット仕様を、すべてXMLタグを使用して行った例**です。

In [ ]:
# First input variable
EMAIL = "Hi Zack, just pinging you for a quick update on that prompt you were supposed to write."

# Second input variable
ADJECTIVE = "olde english"

# Prompt template with a placeholder for the variable content
PROMPT = f"Hey Claude. Here is an email: <email>{EMAIL}</email>. Make this email more {ADJECTIVE}. Write the new version in <{ADJECTIVE}_email> XML tags."

# Prefill for Claude's response (now as an f-string with a variable)
PREFILL = f"<{ADJECTIVE}_email>"

# Print Claude's response
print("--------------------------- 変数置換を含む完全なプロンプト ---------------------------")
print("ユーザーのターン")
print(PROMPT)
print("\nアシスタントのターン")
print(PREFILL)
print("\n------------------------------------- Claudeの応答 -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

#### ボーナスレッスン

APIを通じてClaudeを呼び出す場合、`stop_sequences`パラメータに閉じるXMLタグを渡すことで、Claudeがあなたの望むタグを出力した後にサンプリングを停止させることができます。これにより、Claudeがあなたが気にしている答えをすでに提供した後の結論の発言を排除することで、コストと最後のトークンまでの時間を節約できます。

上記のコンテンツを変更せずにレッスンのプロンプトを試してみたい場合は、レッスンノートブックの一番下までスクロールして[**Example Playground**](#example-playground)にアクセスしてください。

---

## 演習
- [演習 5.1 - ステフィン・カリー GOAT](#exercise-51---steph-curry-goat)
- [演習 5.2 - 二つの俳句](#exercise-52---two-haikus)
- [演習 5.3 - 二つの俳句、二つの動物](#exercise-53---two-haikus-two-animals)

### 演習 5.1 - ステフィン・カリー GOAT
選択を強いられた場合、クロードはマイケル・ジョーダンを史上最高のバスケットボール選手として指名します。クロードに他の選手を選ばせることはできるでしょうか？

`PREFILL`変数を変更して、**クロードに史上最高のバスケットボール選手はステフィン・カリーであるという詳細な議論をさせる**ようにしてください。演習の焦点は`PREFILL`にあるため、それ以外は変更しないようにしてください。

In [ ]:
# プロンプトテンプレートで変数内容のプレースホルダー
PROMPT = f"Who is the best basketball player of all time? Please choose one specific player."

# Claudeの応答のプレフィル
PREFILL = ""

# Claudeの応答を取得
response = get_completion(PROMPT, prefill=PREFILL)

# 演習の正確さを評価する関数
def grade_exercise(text):
    return bool(re.search("Warrior", text))

# Claudeの応答を出力
print("--------------------------- 変数置換を含む完全なプロンプト ---------------------------")
print("ユーザーのターン")
print(PROMPT)
print("\nアシスタントのターン")
print(PREFILL)
print("\n------------------------------------- Claudeの応答 -------------------------------------")
print(response)
print("\n------------------------------------------ 評価 ------------------------------------------")
print("この演習は正しく解決されました:", grade_exercise(response))

ヒントが必要な場合は、以下のセルを実行してください！

In [ ]:
print(hints.exercise_5_1_hint)

### 演習 5.2 - 二つの俳句
以下の`PROMPT`をXMLタグを使用して修正し、Claudeが動物について一つではなく二つの俳句を書くようにします。一つの詩がどこで終わり、もう一つがどこから始まるのかが明確である必要があります。

In [ ]:
# 変数の内容
ANIMAL = ""

# ハイクの数
NUMBER = ""

# Claudeの応答のためのプレフィル
PREFILL = ""

# 変数の内容のためのプレースホルダーを持つプロンプトテンプレート
PROMPT = f"Please write {NUMBER} haiku(s) about {ANIMAL}. Put each haiku(s) in {PREFILL} tags."

# Claudeの応答を取得
response = get_completion(PROMPT, prefill=PREFILL)

# 演習の正確さを評価する関数
def grade_exercise(text):
    return bool(
        (re.search("cat", text.lower()) and re.search("<haiku>", text))
        and (text.count("\n") + 1) > 5
    )

# Claudeの応答を出力
print("--------------------------- 変数の置換を含む完全なプロンプト ---------------------------")
print("ユーザーのターン")
print(PROMPT)
print("\nアシスタントのターン")
print(PREFILL)
print("\n------------------------------------- Claudeの応答 -------------------------------------")
print(response)
print("\n------------------------------------------ 評価 ------------------------------------------")
print("この演習は正しく解決されました:", grade_exercise(response))

ヒントが必要な場合は、以下のセルを実行してください！

In [ ]:
print(hints.exercise_5_2_hint)

### Exercise 5.3 - 二つの俳句、二つの動物
以下の`PROMPT`を修正して、**Claudeが二つの異なる動物についての二つの俳句を作成する**ようにします。最初の置き換えには`{ANIMAL1}`を、二番目の置き換えには`{ANIMAL2}`を使用します。

In [ ]:
# 最初の入力変数
ANIMAL1 = "Cat"

# 2番目の入力変数
ANIMAL2 = "Dog"

# ハイクの数
NUMBER = "two"

# 変数の内容のためのプレースホルダーを持つプロンプトテンプレート
PROMPT = f"Please write a haiku about {ANIMAL1}. Put it in <haiku> tags."

# Claudeの応答を取得
response = get_completion(PROMPT)

# 演習の正確さを評価する関数
def grade_exercise(text):
    return bool(re.search("tail", text.lower()) and re.search("cat", text.lower()) and re.search("<haiku>", text))

# Claudeの応答を出力
print("--------------------------- 変数の置換を含む完全なプロンプト ---------------------------")
print("ユーザーのターン")
print(PROMPT)
print("\n------------------------------------- Claudeの応答 -------------------------------------")
print(response)
print("\n------------------------------------------ 採点 ------------------------------------------")
print("この演習は正しく解決されました:", grade_exercise(response))

ヒントが必要な場合は、以下のセルを実行してください！

In [ ]:
print(hints.exercise_5_3_hint)

### おめでとうございます！

ここまでのすべての演習を解決した場合、次の章に進む準備が整いました。楽しいプロンプト作成を！

---

## 例のプレイグラウンド

これは、このレッスンで示されたプロンプトの例を自由に実験し、プロンプトを調整してClaudeの応答にどのように影響するかを見るためのエリアです。

In [ ]:
# 変数の内容
ANIMAL = "Rabbit"

# 変数の内容のためのプレースホルダーを持つプロンプトテンプレート
PROMPT = f"Please write a haiku about {ANIMAL}. Put it in <haiku> tags."

# Claudeの応答を出力
print("--------------------------- 変数の置換を含む完全なプロンプト ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claudeの応答 -------------------------------------")
print(get_completion(PROMPT))

In [ ]:
# 変数の内容
ANIMAL = "Cat"

# 変数の内容のためのプレースホルダーを持つプロンプトテンプレート
PROMPT = f"Please write a haiku about {ANIMAL}. Put it in <haiku> tags."

# Claudeの応答のための事前入力
PREFILL = "<haiku>"

# Claudeの応答を出力
print("--------------------------- 変数置換を含む完全なプロンプト ---------------------------")
print("ユーザーのターン:")
print(PROMPT)
print("\nアシスタントのターン:")
print(PREFILL)
print("\n------------------------------------- Claudeの応答 -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

In [ ]:
# Variable content
ANIMAL = "Cat"

# プロンプトテンプレート、変数の内容のためのプレースホルダー付き
PROMPT = f"Please write a haiku about {ANIMAL}. Use JSON format with the keys as \"first_line\", \"second_line\", and \"third_line\"."

# Claudeの応答のためのプレフィル
PREFILL = "{"

# Claudeの応答を出力します
print("--------------------------- 変数置換を含む完全なプロンプト ---------------------------")
print("ユーザーのターン")
print(PROMPT)
print("\nアシスタントのターン")
print(PREFILL)
print("\n------------------------------------- Claudeの応答 -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

In [ ]:
# 最初の入力変数
EMAIL = "Hi Zack, just pinging you for a quick update on that prompt you were supposed to write."

# 2番目の入力変数
ADJECTIVE = "olde english"

# 変数の内容のためのプレースホルダーを持つプロンプトテンプレート
PROMPT = f"Hey Claude. Here is an email: <email>{EMAIL}</email>. Make this email more {ADJECTIVE}. Write the new version in <{ADJECTIVE}_email> XML tags."

# Claudeの応答のためのプレフィル（今は変数を持つf-stringとして）
PREFILL = f"<{ADJECTIVE}_email>"

# Claudeの応答を出力
print("--------------------------- 変数置換を含む完全なプロンプト ---------------------------")
print("ユーザーのターン")
print(PROMPT)
print("\nアシスタントのターン")
print(PREFILL)
print("\n------------------------------------- Claudeの応答 -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))